In [1]:
from keras.applications import vgg16

Using TensorFlow backend.


In [2]:
# pre trained weights : create my model
model = vgg16.VGG16(weights='imagenet', include_top = False, input_shape = (240,320,3))

In [3]:
model.layers

In [11]:
for layers in model.layers:
    layers.trainable = False

In [12]:
model.layers[5].trainable

False

In [13]:
model.layers

In [14]:
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense

In [17]:
model.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 10, 512) dtype=float32>

In [18]:
top_model = model.output

In [19]:
top_model

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 10, 512) dtype=float32>

In [20]:
top_model = Flatten()(top_model)

In [21]:
top_model = Dense(512, activation = 'relu')(top_model)

In [22]:
top_model = Dense(256, activation = 'relu')(top_model)

In [23]:
top_model = Dense(2,activation='softmax')(top_model)

In [24]:
model.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 10, 512) dtype=float32>

In [25]:
from keras.models import Model

In [26]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 240, 320, 3) dtype=float32>

In [27]:
top_model

<tf.Tensor 'dense_3/Softmax:0' shape=(None, 2) dtype=float32>

In [28]:
newmodel  = Model(inputs = model.input , outputs = top_model )

In [30]:
newmodel.input

<tf.Tensor 'input_1:0' shape=(None, 240, 320, 3) dtype=float32>

In [31]:
newmodel.output

<tf.Tensor 'dense_3/Softmax:0' shape=(None, 2) dtype=float32>

In [32]:
from keras.preprocessing.image import ImageDataGenerator

In [33]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [34]:
training_set = train_datagen.flow_from_directory(
        'TF_Train/',
        target_size=(240,320),
        batch_size=32,
        class_mode='categorical')

Found 238 images belonging to 2 classes.


In [35]:
test_set = test_datagen.flow_from_directory(
        'TF_Test',
        target_size=(240,320),
        batch_size=32,
        class_mode='categorical')

Found 40 images belonging to 2 classes.


In [36]:
newmodel.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

In [37]:
newmodel.summary

<bound method Network.summary of <keras.engine.training.Model object at 0x0000023A01969448>>

In [38]:
newmodel.fit(
        training_set,
        steps_per_epoch=238,
        epochs=1,
        validation_data=test_set,
        validation_steps=40)

Epoch 1/1
238/238 [==============================] - 3806s 16s/step - loss: 0.0448 - accuracy: 0.9931 - val_loss: 1.1921e-07 - val_accuracy: 1.0000


In [39]:
newmodel.save("vgg_recog.h5")